In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import pickle
import json

In [2]:
df = pd.read_csv("fordTrain.csv")
df.sample(10)

,TrialID,ObsNum,IsAlert,P1,P2,P3,P4,P5,P6,P7,...,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
511541,423,215,1,31.2568,14.50980,1100,54.5455,0.295790,628,95.5414,...,-0.070,496,5.99375,0,1746,0,12.6,0,4,15.02420
477727,395,241,1,31.3933,13.51880,508,118.1100,0.108975,564,106.3830,...,0.000,1008,63.00000,0,717,0,0.0,0,1,11.92290
392414,324,763,1,32.8717,7.63115,1100,54.5455,0.267249,400,150.0000,...,0.070,255,11.98750,0,776,0,0.0,0,1,5.89847
415261,343,614,0,28.4961,11.40840,1208,49.6689,2.292370,404,148.5150,...,0.000,240,1.48750,0,2198,0,23.0,0,4,4.65147
402650,333,113,0,34.2551,15.96890,1180,50.8475,5.255460,632,94.9367,...,-0.385,511,1.48750,1,2202,0,14.5,0,1,11.32430
376705,311,783,1,37.2976,14.99440,800,75.0000,0.264654,520,115.3850,...,-0.070,240,4.50625,0,2042,0,20.4,0,4,16.10180
478774,396,77,1,33.7450,12.14810,600,100.0000,0.094705,616,97.4026,...,0.070,255,3.01875,0,2112,0,24.7,0,4,12.07450
150260,124,396,0,35.5343,11.92340,1060,56.6038,0.084326,848,70.7547,...,0.000,240,7.48125,0,2123,0,25.0,0,4,7.78259
9393,7,928,0,32.7998,14.81060,1200,50.0000,0.108975,672,89.2857,...,0.175,1008,4.50625,0,2342,0,34.1,0,4,16.72200
242577,200,857,1,32.9939,11.53800,900,66.6667,0.277628,560,107.1430,...,0.000,496,1.48750,0,1892,0,24.5,0,4,16.34390


In [3]:
df.describe()

,TrialID,ObsNum,IsAlert,P1,P2,P3,P4,P5,P6,P7,...,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
count,604329.000000,604329.000000,604329.000000,604329.000000,604329.000000,604329.000000,604329.000000,604329.000000,604329.000000,604329.000000,...,604329.000000,604329.000000,604329.000000,604329.000000,604329.000000,604329.0,604329.000000,604329.0,604329.000000,604329.000000
mean,250.167657,603.841765,0.578799,35.449020,11.996525,1026.671035,64.061965,0.178923,845.384610,77.887628,...,-0.037710,573.786433,19.961030,0.179814,1715.688383,0.0,12.710354,0.0,3.312257,11.668277
std,145.446164,348.931601,0.493752,7.484629,3.760292,309.277877,19.755950,0.372309,2505.335141,18.577930,...,0.403896,298.412888,63.269456,0.384033,618.176470,0.0,11.532085,0.0,1.243586,9.934423
min,0.000000,0.000000,0.000000,-22.481200,-45.629200,504.000000,23.885300,0.038920,128.000000,0.262224,...,-4.795000,240.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.000000,1.676730
25%,125.000000,302.000000,0.000000,31.758100,9.903540,792.000000,49.180300,0.092110,668.000000,66.666700,...,-0.175000,255.000000,1.487500,0.000000,1259.000000,0.0,0.000000,0.0,3.000000,7.947680
50%,250.000000,604.000000,1.000000,34.145100,11.400400,1000.000000,60.000000,0.105083,800.000000,75.000000,...,0.000000,511.000000,3.018750,0.000000,1994.000000,0.0,12.800000,0.0,4.000000,10.772600
75%,374.000000,906.000000,1.000000,37.311900,13.644200,1220.000000,75.757600,0.138814,900.000000,89.820400,...,0.070000,767.000000,7.481250,0.000000,2146.000000,0.0,21.900000,0.0,4.000000,15.270900
max,510.000000,1210.000000,1.000000,101.351000,71.173700,2512.000000,119.048000,27.202200,228812.000000,468.750000,...,3.990000,1023.000000,484.488000,1.000000,4892.000000,0.0,82.100000,0.0,7.000000,262.534000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 604329 entries, 0 to 604328
Data columns (total 33 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   TrialID  604329 non-null  int64  
 1   ObsNum   604329 non-null  int64  
 2   IsAlert  604329 non-null  int64  
 3   P1       604329 non-null  float64
 4   P2       604329 non-null  float64
 5   P3       604329 non-null  int64  
 6   P4       604329 non-null  float64
 7   P5       604329 non-null  float64
 8   P6       604329 non-null  int64  
 9   P7       604329 non-null  float64
 10  P8       604329 non-null  int64  
 11  E1       604329 non-null  float64
 12  E2       604329 non-null  float64
 13  E3       604329 non-null  int64  
 14  E4       604329 non-null  int64  
 15  E5       604329 non-null  float64
 16  E6       604329 non-null  int64  
 17  E7       604329 non-null  int64  
 18  E8       604329 non-null  int64  
 19  E9       604329 non-null  int64  
 20  E10      604329 non-null  

In [5]:
df.isna().sum()

TrialID    0
ObsNum     0
IsAlert    0
P1         0
P2         0
P3         0
P4         0
P5         0
P6         0
P7         0
P8         0
E1         0
E2         0
E3         0
E4         0
E5         0
E6         0
E7         0
E8         0
E9         0
E10        0
E11        0
V1         0
V2         0
V3         0
V4         0
V5         0
V6         0
V7         0
V8         0
V9         0
V10        0
V11        0
dtype: int64

In [6]:
df.var()

TrialID    2.115459e+04
ObsNum     1.217533e+05
IsAlert    2.437911e-01
P1         5.601967e+01
P2         1.413979e+01
P3         9.565281e+04
P4         3.902976e+02
P5         1.386137e-01
P6         6.276704e+06
P7         3.451395e+02
P8         0.000000e+00
E1         1.973764e+02
E2         1.619476e+04
E3         1.012362e+00
E4         1.260860e+03
E5         5.308989e-06
E6         7.507585e+02
E7         8.150182e+00
E8         2.588260e+00
E9         1.080315e-01
E10        3.568710e+02
E11        2.753315e+01
V1         1.970208e+03
V2         1.631318e-01
V3         8.905025e+04
V4         4.003024e+03
V5         1.474814e-01
V6         3.821421e+05
V7         0.000000e+00
V8         1.329890e+02
V9         0.000000e+00
V10        1.546507e+00
V11        9.869275e+01
dtype: float64

In [7]:
df1 = df.drop('TrialID', axis=1)
df1.columns

Index(['ObsNum', 'IsAlert', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8',
       'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11',
       'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11'],
      dtype='object')

In [8]:
X = df1.drop('IsAlert', axis=1)
y = df1['IsAlert']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=True,random_state=4)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=0.33, shuffle=True,random_state=4)

In [10]:
print("X_train.shape:", X_train.shape)
print("y_train.shape:", y_train.shape)
print("X_test.shape:", X_test.shape)
print("y_test.shape:", y_test.shape)
print("X_valid.shape:", X_valid.shape)
print("y_valid.shape:", y_valid.shape)

X_train.shape: (404900, 31)
y_train.shape: (404900,)
X_test.shape: (133617, 31)
y_test.shape: (133617,)
X_valid.shape: (65812, 31)
y_valid.shape: (65812,)


In [11]:
scaler = StandardScaler()

In [12]:
ct = make_column_transformer((scaler, ['ObsNum', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8',
       'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11',
       'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11']))

In [15]:
classifiers = [("Logistic Regression", LogisticRegression()), ("Random Forest Classifier", RandomForestClassifier()), 
               ("K Nearest Neighbors", KNeighborsClassifier(n_neighbors=2))]

In [16]:
for clf_name, clf in classifiers:
    pipe = make_pipeline(ct, clf)
    pipe.fit(X_train, y_train)
    train_pred = pipe.predict(X_train)
    train_acc = accuracy_score(y_train, train_pred)
    print("Train accuracy of ", clf_name, "is: ", train_acc)    
    test_pred =pipe.predict(X_test)
    test_acc = accuracy_score(y_test, test_pred)
    print("Test accuracy of ", clf_name, "is: ",test_acc)

Train accuracy of  Logistic Regression is:  0.7901111385527291
Test accuracy of  Logistic Regression is:  0.7916657311569636
Train accuracy of  Random Forest Classifier is:  1.0
Test accuracy of  Random Forest Classifier is:  0.9930772281970108
Train accuracy of  K Nearest Neighbors is:  0.9802815510002469
Test accuracy of  K Nearest Neighbors is:  0.9422079525808842


In [20]:
final_pipe = make_pipeline(ct, RandomForestClassifier())
final_pipe.fit(X_train, y_train)
valid_pred = final_pipe.predict(X_valid)
valid_acc = accuracy_score(y_valid, valid_pred)
print("Accuracy score of Random Forest Classifier on validation test is: ", valid_acc)

Accuracy score of Random Forest Classifier on validation test is:  0.9935726007415061


In [21]:
with open('driver_model.pickle','wb') as f:
    pickle.dump(final_pipe,f)

In [22]:
columns = {
    'data_columns' : [col.lower() for col in df1.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))

In [1]:
import pickle

In [2]:
loaded_model = pickle.load(open('driver_model.pickle', 'rb'))

In [6]:
import pandas as pd
test = pd.read_csv("fordTest.csv")

In [7]:
test1 = test.drop('TrialID', axis=1)

In [8]:
test_preds = loaded_model.predict(test1)

In [9]:
output = pd.DataFrame({'TrialId': test.TrialID,
                       'ObsNum' : test.ObsNum,
                       'Prediction': test_preds})
output.to_csv('submission.csv', index=False)